#Which DNA Patterns Play the Role of Molecular Clocks?

##Code Challenge (1.2 - Step 7)
**Implanted Motif Problem:** *Find all ($k$,$d$)-motifs in a collection of strings.*
* **Input:** A collection of strings `Dna` and integers $k$ and $d$.
* **Output:** All ($k$,$d$)-motifs in `Dna`.

In [6]:
function motif_enumeration(Dna, k, d)
    patterns = Set()
    for string = Dna
        for i = 1:length(string)-k+1
            pattern0 = string[i:i+k-1]
            theNeighbors = neighbors(pattern0, d)
            for pattern1 = theNeighbors
                inAll = true
                for string1 = Dna
                    inString = false
                    for j = 1:length(string1)-k+1
                        pattern2 = string1[j:j+k-1]
                        if hamming(pattern1, pattern2) <= d
                            inString = true
                        end
                    end
                    if !inString
                        inAll = false
                    end
                end
                if inAll
                    push!(patterns, pattern1)
                end
            end
        end
    end
    return patterns
end

motif_enumeration (generic function with 1 method)

In [7]:
function neighbors(pattern, d)
    if d == 0
        return Set([pattern])
    end
    if length(pattern) == 1
        return Set(['A', 'C', 'G', 'T'])
    end
    neighborhood = Set{ASCIIString}()
    suffixNeighbors = neighbors(pattern[2:end], d)
    for text = suffixNeighbors
        if hamming(pattern[2:end], text) < d
            for x = Set(['A', 'C', 'G', 'T'])
                push!(neighborhood, string(x, text))
            end
        elseif hamming(pattern[2:end], text) == d
            push!(neighborhood, string(first(pattern), text))
        end
    end
    return collect(neighborhood)
end

neighbors (generic function with 1 method)

In [8]:
function hamming(string1, string2)
    distance = 0
    for i = 1:length(string1)
        if string1[i] != string2[i]
            distance += 1
        end
    end
    return distance
end

hamming (generic function with 1 method)

In [9]:
Dna = ["ATTTGGC","TGCCTTA","CGGTATC","GAAAATT"]
k = 3
D = 1
motif_enumeration(Dna, k, D)

Set(Any["ATT","TTT","ATA","GTT"])

In [10]:
data = readdlm("data/dataset_156_7.txt")
k = data[1,1]
D = data[1,2]
Dna = data[2:end,1]
output = motif_enumeration(Dna, k, D)
writedlm("data/output_156_7.txt", output)

##Code Challenge (1.4 - Step 9)
**Median String Problem:** *Find a median string.*
* **Input:** An integer $k$, followed by a collection of strings `Dna`.
* **Output:** A $k$-mer `pattern` that minimizes $d$(`pattern`, `Dna`) among all $k$-mers `pattern`.

In [11]:
function median_string(Dna, k)
    distance = Inf
    median = ""
    for i = 1:4^k
        pattern = number_to_pattern(i-1, k)
        newDist = d(pattern, Dna)
        if newDist < distance
            distance = newDist
            median = pattern
        end
    end
    return median
end

median_string (generic function with 1 method)

In [12]:
function d{T<:AbstractString}(pattern, text::T)
    k = length(pattern)
    distance = Inf
    for i = 1:length(text)-k+1
        kmer = text[i:i+k-1]
        newDist = hamming(kmer, pattern)
        if newDist < distance
            distance = newDist
        end
    end
    return distance
end

d (generic function with 1 method)

In [13]:
function d{T<:AbstractString}(pattern, Dna::Array{T,1})
    t = length(Dna)
    totalDist = 0
    for i = 1:t
        totalDist += d(pattern, Dna[i])
    end
    return totalDist
end

d (generic function with 2 methods)

In [14]:
function number_to_pattern(index, k)
    if k == 1
        return number_to_symbol(index)
    else
        prefixIndex = div(index, 4)
        r = rem(index, 4)
        symbol = number_to_symbol(r)
        prefixPattern = number_to_pattern(prefixIndex, k-1)
        return string(prefixPattern, symbol)
    end
end

number_to_pattern (generic function with 1 method)

In [15]:
function number_to_symbol(k)
    dict = Dict(0 => 'A', 1 => 'C', 2 => 'G', 3 => 'T')
    return dict[k]
end

number_to_symbol (generic function with 1 method)

In [16]:
f = open("data/dataset_158_9.txt")
k = parse(strip(readline(f)))
Dna = convert(Array{ASCIIString}, readdlm(f)[:,1])
close(f)
median_string(Dna, k)

"CTCCAG"

##Code Challenge (1.5 - Step 3)
***Profile*-most probable $k$-mer problem:** *Find a Profile-most probable $k$-mer in a string.*
* **Input:** A string `text`, and integer $k$, and a $4 \times k$ matrix `profile`.
* **Output:** A `profile`-most probable $k$-mer in `text`.

In [17]:
function most_probable_kmer(text, k, profile)
    score = -Inf
    bestKmer = ""
    for i = 1:length(text)-k+1
        kmer = text[i:i+k-1]
        newScore = prob(kmer, profile)
        if newScore > score
            score = newScore
            bestKmer = kmer
        end
    end
    return bestKmer
end

most_probable_kmer (generic function with 1 method)

In [18]:
function prob(kmer, profile)
    score = 1
    dict = Dict('A' => 0, 'C' => 1, 'G' => 2, 'T' => 3)
    for j = 1:length(kmer)
        i = dict[kmer[j]] + 1
        score *= profile[i,j]
    end
    return score
end

prob (generic function with 1 method)

In [19]:
function symbol_to_number(symbol)
    dict = Dict('A' => 0, 'C' => 1, 'G' => 2, 'T' => 3)
    return dict[symbol]
end

symbol_to_number (generic function with 1 method)

In [20]:
f = open("data/dataset_159_3.txt")
text = strip(readline(f))
k = parse(strip(readline(f)))
profile = readdlm(f)
close(f)
most_probable_kmer(text, k, profile)

"CATGGATTGAGT"

##Code Challenge (1.5 - Step 5)
* **Input:** Integers $k$ and $t$, followed by a collection of strings `Dna`.
* **Output:** A collection of strings `bestMotifs` resulting from applying `greedy_motif_search(Dna`, $k$, $t$`)`.

In [21]:
function greedy_motif_search(Dna, k, t)
    bestMotifs = []
    for i = 1:t
        push!(bestMotifs, Dna[i][1:k])
    end
    motifs = Array(ASCIIString, t)
    for j = 1:length(Dna[1])-k+1
        motif = Dna[1][j:j+k-1]
        motifs[1] = motif
        for i = 2:t
            profile = make_profile(motifs[1:i-1])
            motifs[i] = most_probable_kmer(Dna[i], k, profile)
        end
        if score(motifs) < score(bestMotifs)
            bestMotifs = motifs[:]
        end
    end
    return bestMotifs
end

greedy_motif_search (generic function with 1 method)

In [22]:
function make_profile(motifs)
    m = length(motifs)
    n = length(motifs[1])
    profile = Array(Float64, 4, n)
    for j = 1:n
        letterCounts = zeros(4) # A, C, G,T
        for i = 1:m
            letterCounts[symbol_to_number(motifs[i][j]) + 1] += 1
        end
        letterFreqs = letterCounts / sum(letterCounts)
        profile[:,j] = letterFreqs
    end
    return profile
end

make_profile (generic function with 1 method)

In [23]:
function score(motifs)
    m = length(motifs)
    n = length(motifs[1])
    totalScore = 0
    for j = 1:n
        letterCounts = zeros(4)
        for i = 1:m
            letterCounts[symbol_to_number(motifs[i][j]) + 1] += 1
        end
        maxnum = maximum(letterCounts)
        totalScore += m - maxnum
    end
    return totalScore
end

score (generic function with 1 method)

In [24]:
motifs = ["TCGGGGGTTTTT",
"CCGGTGACTTAC",
"ACGGGGATTTTC",
"TTGGGGACTTTT",
"AAGGGGACTTCC",
"TTGGGGACTTCC",
"TCGGGGATTCAT",
"TCGGGGATTCCT",
"TAGGGGAACTAC",
"TCGGGTATAACC"]
score(motifs) # test case for score()
make_profile(motifs) # test case for make_profile

4x12 Array{Float64,2}:
 0.2  0.2  0.0  0.0  0.0  0.0  0.9  0.1  0.1  0.1  0.3  0.0
 0.1  0.6  0.0  0.0  0.0  0.0  0.0  0.4  0.1  0.2  0.4  0.6
 0.0  0.0  1.0  1.0  0.9  0.9  0.1  0.0  0.0  0.0  0.0  0.0
 0.7  0.2  0.0  0.0  0.1  0.1  0.0  0.5  0.8  0.7  0.3  0.4

In [25]:
Dna = ["GGCGTTCAGGCA",
"AAGAATCAGTCA",
"CAAGGAGTTCGC",
"CACGTCAATCAC",
"CAATAATATTCG"]
k = 3
t = 5
greedy_motif_search(Dna, k, t)

5-element Array{ASCIIString,1}:
 "CAG"
 "CAG"
 "CAA"
 "CAA"
 "CAA"

In [26]:
f = open("data/dataset_159_5.txt")
nums = split(readline(f))
k = parse(nums[1])
t = parse(nums[2])
Dna = readdlm(f)
close(f)
greedy_motif_search(Dna, k, t)

25-element Array{ASCIIString,1}:
 "GATCGTTTCCCA"
 "ACCCCACCTCCG"
 "AATTAGCTAAGG"
 "CGGCCCGCGCGA"
 "CCCCCAGTAACA"
 "CCTTCGGCAAGA"
 "CGGCCTTCGCGA"
 "CGGCCCGCTCGA"
 "CGGCCGTTGCGA"
 "CGGCCTCCTCGA"
 "CGGCCCTTTCGA"
 "CGGCCCCAGCGA"
 "GCGCCATAACCA"
 "CGGCCTTCCCGA"
 "CGCTATGTAACG"
 "CGGCCGCATCGA"
 "AGGCGACCGCGA"
 "CGGCCACCACGA"
 "GGCCCCCAAAGA"
 "CGGCCACCGCGA"
 "CGGCCTGCACGA"
 "GGTTCGGCGCCA"
 "GGTCGACCCCCA"
 "CGGCCACAACGA"
 "CGGCCACAACGA"

##Code Challenge (1.6 - Step 9)
* **Input:** Integers $k$ and $t$, followed by a collection of strings `Dna`.
* **Output:** A collection of strings `bestMotifs` resulting from applying `greedy_motif_search(Dna`, $k$, $t$`)` with pseudocounts.

In [27]:
function greedy_motif_search_pseudocount(Dna, k, t)
    bestMotifs = []
    for i = 1:t
        push!(bestMotifs, Dna[i][1:k])
    end
    motifs = Array(ASCIIString, t)
    for j = 1:length(Dna[1])-k+1
        motif = Dna[1][j:j+k-1]
        motifs[1] = motif
        for i = 2:t
            profile = make_profile_pseudocount(motifs[1:i-1])
            motifs[i] = most_probable_kmer(Dna[i], k, profile)
        end
        if score(motifs) < score(bestMotifs)
            bestMotifs = motifs[:]
        end
    end
    return bestMotifs
end

greedy_motif_search_pseudocount (generic function with 1 method)

In [28]:
function make_profile_pseudocount(motifs)
    m = length(motifs)
    n = length(motifs[1])
    profile = Array(Float64, 4, n)
    for j = 1:n
        letterCounts = zeros(4) # A, C, G,T
        for i = 1:m
            letterCounts[symbol_to_number(motifs[i][j]) + 1] += 1
        end
        letterCounts += 1 # pseudocount of 1
        letterFreqs = letterCounts / sum(letterCounts)
        profile[:,j] = letterFreqs
    end
    return profile
end

make_profile_pseudocount (generic function with 1 method)

In [29]:
f = open("data/dataset_160_9.txt")
nums = split(readline(f))
k = parse(nums[1])
t = parse(nums[2])
Dna = readdlm(f)
close(f)
greedy_motif_search_pseudocount(Dna, k, t)

25-element Array{ASCIIString,1}:
 "ACTCTGACTATC"
 "ACTCGGACTATG"
 "ACTCGGACTATT"
 "ACTCAGGCTCTC"
 "ACTCCGGCTCTA"
 "ACTCGGACTATG"
 "ACTCCGCCTCTG"
 "ACTCGGACTGTT"
 "ACTCCGTCTGTT"
 "ACTCGGCCTTTG"
 "ACTCAGACTATT"
 "ACTCTGCCTCTA"
 "ACTCGGGCTCTT"
 "ACTCCGTCTTTA"
 "ACTCAGTCTCTA"
 "ACTCGGCCTTTA"
 "ACTCGGCCTTTT"
 "ACTCCGCCTATT"
 "ACTCAGTCTATA"
 "ACTCCGCCTTTA"
 "ACTCAGGCTCTC"
 "ACTCGGGCTTTA"
 "ACTCCGCCTGTT"
 "ACTCCGTCTTTA"
 "ACTCTGCCTCTG"

##Code Challenge (2.1 - Step 5)
###Randomized Motif Search
* **Input:** Integers $k$ and $t$, followed by a collection of strings `Dna`.
* **Output:** A collection `bestMotifs` resulting from running `randomized_motif_search(Dna`, $k$, $t$`)` one hundred times.

In [30]:
function randomized_motif_search1000(Dna, k, t)
    bestMotifs = []
    bestScore = Inf
    for i = 1:1000
        motifs, score = randomized_motif_search(Dna, k, t)
        if score < bestScore
            bestMotifs = motifs[:]
            bestScore = score
        end
    end
    return bestMotifs
end

randomized_motif_search1000 (generic function with 1 method)

In [31]:
function randomized_motif_search(Dna, k, t)
    motifs = []
    for i = 1:t
        possibleStarts = length(Dna[i]) - k + 1
        randStart = convert(Int64, ceil(possibleStarts * rand()))
        push!(motifs, Dna[i][randStart:randStart+k-1])
    end
    bestMotifs = motifs[:]
    bestScore = Inf
    scoreLookup = Dict()
    while true
        profile = make_profile_pseudocount(motifs)
        motifs = most_probable_motifs(Dna, k, profile)
        currScore, scoreLookup = score_fast(motifs, scoreLookup)
        if currScore < bestScore
            bestMotifs = motifs[:]
            bestScore = currScore
        else
            return bestMotifs, bestScore
        end
    end
end

randomized_motif_search (generic function with 1 method)

In [32]:
function most_probable_motifs(Dna, k, profile)
    motifs = []
    for i = 1:length(Dna)
        push!(motifs, most_probable_kmer(Dna[i], k, profile))
    end
    return motifs
end

most_probable_motifs (generic function with 1 method)

In [5]:
Dna = ["AAGCCAAA",
"AATCCTGG",
"GCTACTTG",
"ATGTTTTG"]
motifs = ["CCA",
"CCT",
"CTT",
"TTG"]
most_probable_motifs(Dna, 3, make_profile_pseudocount(motifs))

LoadError: LoadError: UndefVarError: symbol_to_number not defined
while loading In[5], in expression starting on line 9

In [72]:
function score_fast(motifs, scoreLookup)
    if !haskey(scoreLookup, motifs)
        m = length(motifs)
        n = length(motifs[1])
        totalScore = 0
        for j = 1:n
            letterCounts = zeros(4)
            for i = 1:m
                letterCounts[symbol_to_number(motifs[i][j]) + 1] += 1
            end
            maxnum = maximum(letterCounts)
            totalScore += m - maxnum
        end
        scoreLookup[motifs] = totalScore
    end
    return scoreLookup[motifs], scoreLookup
end

score_fast (generic function with 1 method)

In [74]:
f = open("data/dataset_161_5.txt")
nums = split(readline(f))
k = parse(nums[1])
t = parse(nums[2])
Dna = readdlm(f)
close(f)
randomized_motif_search1000(Dna, k, t)

 28

20-element Array{Any,1}:
 "TATCACATAAATTAA"
 "TATTGTACGGACAAA"
 "TATCCAACGGATTGA"
 "TATCGGGCGGACAAA"
 "TATCCAACCACCAAA"
 "TGCACAACGGACAAA"
 "TATCCAACGGACCTC"
 "TATGGTACGGACAAA"
 "CTTCCAACGGACAAC"
 "TATCACTCGGACAAA"
 "TATCCTCTGGACAAA"
 "TATCCACATGACAAA"
 "GATCCAACGGACATC"
 "TATCCACAAGACAAA"
 "TATCCAACGCTTAAA"
 "TACGGAACGGACAAA"
 "ATACCAACGGACAAA"
 "TATCCCTGGGACAAA"
 "TATCCAACGGTACAA"
 "TATCCAAAAAACAAA"

.905479 seconds (596.57 M allocations: 16.196 GB, 13.85% gc time)


## Code Challenge (2.3 - Step 4)
### Gibbs Sampler
* **Input:** Integers $k$, $t$, and $N$, followed by a collection of strings `Dna`.
* **Output:** The strings `bestMotifs` resulting from running `gibbs_sampler(Dna`, $k$, $t$, $N$`)` with 20 random starts.

In [106]:
function gibbs_sampler20(Dna, k, t, N)
    bestMotifs = []
    bestScore = Inf
    for i = 1:20
        motifs, score = gibbs_sampler(Dna, k, t, N)
        if score < bestScore
            bestMotifs = motifs[:]
            bestScore = score
        end
    end
    return bestMotifs
end

gibbs_sampler20 (generic function with 1 method)

In [107]:
function gibbs_sampler(Dna, k, t, N)
    motifs = []
    for i = 1:t
        possibleStarts = length(Dna[i]) - k + 1
        randStart = convert(Int64, ceil(possibleStarts * rand()))
        push!(motifs, Dna[i][randStart:randStart+k-1])
    end
    bestMotifs = motifs[:]
    for j = 1:N
        i = convert(Int64, ceil(t * rand()))
        profile = make_profile_pseudocount(vcat(motifs[1:i-1],motifs[i+1:end]))
        motifs[i] = generate_kmer(profile, Dna[i])
        if score(motifs) < score(bestMotifs)
            bestMotifs = motifs[:]
        end
    end
    return bestMotifs, score(bestMotifs)
end

gibbs_sampler (generic function with 1 method)

In [108]:
using StatsBase
function generate_kmer(profile, text)
    p = Array(Float64, length(text)-k+1)
    for i = 1:length(text)-k+1
        pattern = text[i:i+k-1]
        p[i] = prob(pattern, profile)
    end
    p /= sum(p)
    index = sample(1:length(text)-k+1, WeightVec(p))
    return text[index:index+k-1]
end

generate_kmer (generic function with 1 method)

In [91]:
Dna = ["CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
k = 8
t = 5
N = 100
gibbs_sampler(Dna, k, t, N)

5-element Array{Any,1}:
 "TCTCGGGG"
 "CCAAGGTG"
 "TACAGGCG"
 "TTCAGGTG"
 "TCCACGTG"

In [110]:
f = open("data/dataset_163_4.txt")
nums = split(readline(f))
k = parse(nums[1])
t = parse(nums[2])
N = parse(nums[3])
Dna = readdlm(f)
close(f)
@time gibbs_sampler20(Dna, k, t, N)

 61

20-element Array{Any,1}:
 "CTTGTAGCTGAACAT"
 "CCACGAGGTGGACAT"
 "CCAGCTGGTTAGCAT"
 "ACAGCTGGTGGACCA"
 "CCAGCTGGAAAACAT"
 "CCAGCTGGTGGCTCT"
 "CCAGCACCTGGACAT"
 "CCCTTTGGTGGACAT"
 "CCAGCTGGTGTTAAT"
 "CTCACTGGTGGACAT"
 "CCAGCTGACTGACAT"
 "CCAGAACGTGGACAT"
 "CCAGGGAGTGGACAT"
 "CCAGCCCATGGACAT"
 "CCAGCTGGTGGAGCC"
 "AAAGCTGGTGGACAG"
 "AACGCTGGTGGACAT"
 "CCAAGGGGTGGACAT"
 "CCAGCTCAGGGACAT"
 "CCAGCTATAGGACAT"

.716299 seconds (743.05 M allocations: 31.230 GB, 12.43% gc time)
